<a href="https://colab.research.google.com/github/p4palu1/pythonProject/blob/main/eye_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Magic box`


#**`Warning !!!!!!!!`**
```
## Don't mess with my code ✊
```



#**`I was serious it hasn't recovery option`**

Imports

In [ ]:
import cv2
import numpy as np
import csv

create variables

In [ ]:
# to do methos instead long code

cap = cv2.VideoCapture("eye_recording.flv")
# eye detection tool
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

## variables for average eye area calculation
sum_areas = 0
count_frames = 0
average = 0 

#point of center
X0,Y0=0,0
X_values, Y_values=np.array([]).reshape(-1, 1), np.array([]).reshape(-1, 1)
#each block_size will be data analisis
frames_block_size=2 #sampling resolution
# the number of division areas of the angle
div_num = 8

#num of common moevments to export
moevments_to_export = 4

#noise ignore level at the movements
noise_ignore_level=1

NameError: ignored

Data Mining

In [ ]:

with open('angle and length of eye.csv', 'w', newline='') as csvfile:
    fieldnames = ['angle', 'length','T','F']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()


    while True:
        ret, frame = cap.read()
        if ret is False:
            break
    
        roi = frame[269: 795, 637: 1516]
        rows, cols, _ = roi.shape
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray_roi = cv2.GaussianBlur(gray_roi, (7, 7), 0)
    
        ret, threshold = cv2.threshold(gray_roi, 3, 255, cv2.THRESH_BINARY_INV)
        contours, hierarchy  = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

        ## implementing the cascade
        ##eyes = eye_cascade.detectMultiScale(gray_roi)
        ##biggest = 0

        ## The program usually detects multiple eyes(incorrectly), this code takes the most correct detection
        ##for (ex, ey, ew, eh) in eyes:
        ##    area = (ew-ex)*(eh-ey)
        ##    if area > biggest:
        ##        biggest = area

        ##for (ex, ey, ew, eh) in eyes:
        ##    area = (ew-ex)*(eh-ey)
        ##    if area == biggest:
        ##        cv2.rectangle(roi, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

        ##sum_areas += biggest
        ##count_frames += 1
        ##average = 0 
        ##if count_frames > 0:
        ##  average = sum_areas/count_frames

        for cnt in contours:
            (x, y, w, h) = cv2.boundingRect(cnt)

            #save start point (assume is center point of eyes)
            if(X0==0):
                X0+=x
                Y0+=y

            #add the frame point
            X_values=np.append(X_values,x-X0).reshape(-1, 1)
            Y_values=np.append(Y_values,y-Y0).reshape(-1, 1)
            cv2.drawContours(roi, [cnt], -1, (0, 0, 255), 3)
            cv2.rectangle(roi, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.line(roi, (x + int(w/2), 0), (x + int(w/2), rows), (0, 255, 0), 2)
            cv2.line(roi, (0, y + int(h/2)), (cols, y + int(h/2)), (0, 255, 0), 2)
            break
    
        if(X_values.size==frames_block_size):

            # SVR for X and Y any analysis_each_frames
            from sklearn.preprocessing import StandardScaler
            sc_X = StandardScaler()
            sc_y = StandardScaler()
            X = sc_X.fit_transform(X_values)
            y = sc_y.fit_transform(Y_values)
            from sklearn.svm import SVR
            regressor = SVR(kernel = 'rbf')
            regressor.fit(X, y)

            import math
            # get the angle from the gradiant
            angle = math.atan(sc_y.inverse_transform(regressor.predict(np.array([1]).reshape(-1, 1)).reshape(-1,1)))
            if(angle<0):
                angle=2+angle
            angle=angle*180//(360/div_num)

            #get length
            lenY= sc_y.inverse_transform(regressor.predict(sc_X.transform([[frames_block_size]])).reshape(-1,1))*0.5 #reducing impact of Y on the length
            length = math.sqrt(math.pow(frames_block_size, 2)+math.pow(lenY, 2))//10 #accuracy
            X_values, Y_values=np.array([]), np.array([])
            
            writer.writerow({'angle': angle, 'length': length,'T':angle<div_num/2,'F':angle>=div_num/2})
##print(average)

Data analysis

In [ ]:
with open('angle and length of eye.csv', mode='r') as infile:
    reader = csv.reader(infile)        
    header = next(reader)

    # count frequency of movements
    lastvalue=-1
    noise_level=0
    index=-1
    movements_index_names= list()
    from collections import Counter
    movements_frequency=Counter()
    for rows in reader:
      if(float(rows[1])>0):
        if(lastvalue==-1):
          lastvalue=rows[0]
        if(lastvalue!=rows[0] and noise_level<noise_ignore_level):
            noise_level+=1
        else:
          noise_level=0
          if(index==-1 or lastvalue!=rows[0]):
            lastvalue=rows[0]
            index+=1
            movements_index_names.append(float(rows[0]))
            movements_frequency[index]=0
          movements_frequency[index]+=1
    
    most_common = movements_frequency.most_common(moevments_to_export)
    
    most_common.sort()
    

    #write results
    with open('movements.csv', mode='w') as outfile:
      fieldnames=list()
      for i in range(0,moevments_to_export):
        fieldnames.append('movement '+ str(i))
        
      fieldnames.append('T')
      fieldnames.append('F')
      row=dict()
      i=0
      for common in most_common:
        row[fieldnames[i]]=movements_index_names[common[0]]
        print(row)
        i+=1
      row['T']=0
      row['F']=1
      writer = csv.DictWriter(outfile, fieldnames=fieldnames)
      writer.writeheader()
      writer.writerow(row)


{'movement 0': 1.0}
{'movement 0': 1.0, 'movement 1': 6.0}
{'movement 0': 1.0, 'movement 1': 6.0, 'movement 2': 1.0}
